## 4D Objects By Change - Analysis

This notebook will explain you how an analysis using the *4D Objects By Change* algorithm is run in `py4dgeo`. For details about the algorithm, we refer to the articles *TODO: Instert Articles*

In [ ]:
import py4dgeo

We again trigger a dynamic fetch of the test data for this notebook from the [py4dgeo test data repository](https://github.com/ssciwr/py4dgeo-test-data/):

In [ ]:
py4dgeo.ensure_test_data_availability()

The necessary data for the analysis is stored in an analysis file. There is a dedicated [notebook on creating these analysis files](4dobc-creation.ipynb). In this notebooks, we assume that the file already exists and contains data about a space-time analysis. You can pass an absolute path to the analysis class or have `py4dgeo` locate files for relative paths:

In [ ]:
analysis = py4dgeo.SpatiotemporalAnalysis("synthetic.zip")

If needed, the data can be retrieved from the analysis object. Note that this is a lazy operation, meaning that only explicitly requesting such data will trigger the retrieval from disk into memory:

In [ ]:
analysis.distances

Next, we will construct an algorithm object. You can do so by simply instatiating the `RegionGrowingAlgorithm`:

In [ ]:
algo = py4dgeo.RegionGrowingAlgorithm(neighborhood_radius=2.0, seed_subsampling=30)

The `neighborhood_radius` parameter for this algorithm is of paramount importance: At each core point, a radius search with this radius is performed to determine which other core points are considered to be direct neighbors. This allows us to perform a region growing algorithm on a fully unstructured set of core points. The `seed_subsampling` parameter is used to speed up the generation of seed candidates by only investigating every 30th core point for change points in that core point's timeseries. This is only done to improve the usability of this notebook - for production runs you should omit that parameter (using its default of `1`).

Next, we execute the algorithm on our `analysis` object:

In [ ]:
objects = algo.run(analysis)

The algorithm returns a list of *Objects by change* (grown regions in the space time domain) which we can e.g. plot interactively:

In [ ]:
objects[0].plot()

Similarly to how the `M3C2` class can be customized by subclassing from it, it is possible to create subclasses of `RegionGrowingAlgorithm` to customize some aspects of the region growing algorithm (although this is not covered in detail in this notebook).